### Explaining high balanced accuracy score

In [32]:
print(single_step_window.test_df[target_column].value_counts(normalize=False))
print(single_step_window.test_df[target_column].value_counts(normalize=True))

0    160
1     13
Name: wearing_off, dtype: int64
0    0.924855
1    0.075145
Name: wearing_off, dtype: float64


#### Binary 

In [114]:
m_bin = tf.keras.metrics.BinaryAccuracy()
m_bal = BalancedSparseCategoricalAccuracy()
m_auc = tf.keras.metrics.AUC()

y_trues = []
y_preds = []
for input, label in single_step_window.test(REMOVE_WEARING_OFF_IN_PREVIOUS_STEP):
    y_true = label
    y_trues.append(y_true.numpy())
    print(f'Label: { [y_t for y_t in y_true.numpy().reshape(y_true.numpy().shape[0]) ]}')
    
    y_pred = linear(input)
    y_preds.append(y_pred.numpy())
    print(f'Predictions: {  [y_p for y_p in y_pred.numpy().reshape(y_pred.numpy().shape[0]) ]}')
    
    THRESHOLD = 0.5
    cm = confusion_matrix(y_true.numpy().reshape(y_true.shape[0]),
                          y_pred.numpy().reshape(y_pred.shape[0]) > THRESHOLD)
    print(cm)
    
    m_bin.update_state(y_true, y_pred)
    m_bal.update_state(y_true, y_pred)
    m_auc.update_state(y_true, y_pred)
    print(f'bin result: {m_bin.result().numpy()}')
    print(f'bal result: {m_bal.result().numpy()}')
    print(f'auc result: {m_auc.result().numpy()}')
    print()
    
# y_trues = np.concatenate(y_trues)
# y_preds = np.concatenate(y_preds)

Label: [0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0]
Predictions: [0.0076432526, 0.007150322, 0.007214129, 0.0071855783, 0.0072901547, 0.007442087, 0.0075894594, 0.007887065]
[[6 0]
 [2 0]]
bin result: 0.75
bal result: 0.5
auc result: 0.5

Label: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Predictions: [7.246245e-05, 7.3380616e-05, 7.232216e-05, 7.4307514e-05, 8.272228e-05, 0.00011521306, 7.449562e-05, 7.554495e-05]
[[8]]
bin result: 0.875
bal result: 0.6666666865348816
auc result: 0.7857142686843872

Label: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Predictions: [9.30163e-05, 7.140845e-05, 7.2642004e-05, 7.38648e-05, 7.50703e-05, 7.625169e-05, 7.7401965e-05, 7.851433e-05]
[[8]]
bin result: 0.9166666865348816
bal result: 0.75
auc result: 0.8636363744735718

Label: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Predictions: [0.011505932, 0.00753358, 0.010426879, 0.0075974166, 0.007124424, 0.007491082, 0.0070338547, 0.0070899427]
[[8]]
bin result: 0.9375
bal result: 0.800000011920929
auc result: 0.73333

In [117]:
m_bal.count

<tf.Variable 'count:0' shape=() dtype=float32, numpy=2.0>

In [207]:
m_bin = tf.keras.metrics.BinaryAccuracy()
m_bal = BalancedSparseCategoricalAccuracy()
m_auc = tf.keras.metrics.AUC()

m_bin.update_state(tf.convert_to_tensor( np.concatenate(y_trues) ), tf.convert_to_tensor( np.concatenate(y_preds) ))
m_bal.update_state(tf.convert_to_tensor( np.concatenate(y_trues) ), tf.convert_to_tensor( np.concatenate(y_preds) ))
m_auc.update_state(tf.convert_to_tensor( np.concatenate(y_trues) ), tf.convert_to_tensor( np.concatenate(y_preds) ))
print(f'bin result: {m_bin.result().numpy()}')
print(f'bal result: {m_bal.result().numpy()}')
print(f'auc result: {m_auc.result().numpy()}')

bin result: 0.9230769276618958
bal result: 0.5
auc result: 0.5638560056686401


In [195]:
m = tf.keras.metrics.BinaryAccuracy()
m.update_state([0, 0, 0, 0, 1, 1, 0, 0], 
               [0.007643253, 0.007643253, 0.007150322, 0.007214129, 0.007185578, 0.007290155, 0.007442087, 0.007589459])
print('Keras Accuracy')
print('Final result: ', m.result().numpy())  # Final result: 0.75

Keras Accuracy
Final result:  0.75


In [196]:
m = tf.keras.metrics.BinaryAccuracy()
m.update_state([[1], [1], [0], [0]], 
               [[0.98], [1], [0], [0.6]],
               sample_weight=[1, 0, 0, 1]
              )
print('Keras Balanced Accuracy')
print('Final result: ', m.result().numpy())  # Final result: 0.75

Keras Balanced Accuracy
Final result:  0.5


In [202]:
m = tf.keras.metrics.BinaryAccuracy()
m.update_state(np.array([0, 0, 0, 0, 1, 1, 0, 0]).reshape(8,1), 
               np.array([0.007643253, 0.007643253, 0.007150322, 0.007214129, 0.007185578, 0.007290155, 0.007442087, 0.007589459]).reshape(8,1),
               sample_weight=[0.166666667, 0.166666667, 0.166666667, 0.166666667, 0.5, 0.5, 0.166666667, 0.166666667]
              )
print('Keras Balanced Accuracy')
print('Final result: ', m.result().numpy())  # Final result: 0.75

Keras Balanced Accuracy
Final result:  0.5


In [203]:
print('Sci-kit Accuracy')
print(accuracy_score([0, 0, 0, 0, 1, 1, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0]))

Sci-kit Accuracy
0.75


In [204]:
print('Sci-kit Accuracy')
print(accuracy_score([0, 0, 0, 0, 1, 1, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0],
                     sample_weight=[0.166666667, 0.166666667, 0.166666667, 0.166666667, 0.5, 0.5, 0.166666667, 0.166666667]
                    ))

Sci-kit Accuracy
0.5000000004999999


In [205]:
print('Sci-kit Balanced Accuracy')
print(balanced_accuracy_score([0, 0, 0, 0, 1, 1, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0]))

Sci-kit Balanced Accuracy
0.5


In [212]:
m = tf.keras.metrics.BinaryAccuracy()
m.update_state([0], [0.007643253], sample_weight=[[0.166666667]])
print('Final result: ', m.result().numpy())  # Final result: 0.75

m.update_state([0], [0.007643253], sample_weight=[[0.166666667]])
print('Final result: ', m.result().numpy())  # Final result: 0.75

m.update_state([0], [0.007150322], sample_weight=[[0.166666667]])
print('Final result: ', m.result().numpy())  # Final result: 0.75

m.update_state([0], [0.007214129], sample_weight=[[0.166666667]])
print('Final result: ', m.result().numpy())  # Final result: 0.75

m.update_state([1], [0.007185578], sample_weight=[[0.5]])
print('Final result: ', m.result().numpy())  # Final result: 0.75

m.update_state([1], [0.007290155], sample_weight=[[0.5]])
print('Final result: ', m.result().numpy())  # Final result: 0.75

m.update_state([0], [0.007442087], sample_weight=[[0.166666667]])
print('Final result: ', m.result().numpy())  # Final result: 0.75

m.update_state([0], [0.007589459], sample_weight=[[0.166666667]])
print('Final result: ', m.result().numpy())  # Final result: 0.75

Final result:  1.0
Final result:  1.0
Final result:  1.0
Final result:  1.0
Final result:  0.57142854
Final result:  0.4
Final result:  0.45454547
Final result:  0.5


In [108]:
weights = []
for i in np.concatenate(y_trues).reshape(169):
    if i == 0:
        weights.append(160/173)
    else:
        weights.append(13/173)

In [110]:
from keras.utils import metrics_utils
from keras.utils import losses_utils
from keras.metrics import MeanMetricWrapper
from keras.metrics import Reduce

In [111]:
m = tf.keras.metrics.BinaryAccuracy()
r = Reduce(
    MeanMetricWrapper(m._fn).reduction,
    MeanMetricWrapper(m._fn).name
)
total = r.total
count = r.count

# https://github.com/keras-team/keras/blob/07e13740fd181fc3ddec7d9a594d8a08666645f6/keras/metrics/base_metric.py#L616
# update_state
y_true = y_trues[0]
y_pred = y_preds[0]
sample_weight = weights[0]
threshold = 0.5

y_true = tf.cast(y_true, 'float32')
y_pred = tf.cast(y_pred, 'float32')
[y_true, y_pred], sample_weight = (
    metrics_utils.ragged_assert_compatible_and_get_flat_values(
        [y_true, y_pred], sample_weight))
y_pred, y_true = losses_utils.squeeze_or_expand_dimensions(
    y_pred, y_true)

ag_fn = tf.__internal__.autograph.tf_convert(m._fn, tf.__internal__.autograph.control_status_ctx())
matches = ag_fn(y_true, y_pred, **m._fn_kwargs)
values = matches

In [ ]:
# Explainer on how to get matches / values using ag_fn's binary_accuracy function
# https://github.com/keras-team/keras/blob/v2.9.0/keras/metrics/metrics.py#L3240
# https://github.com/keras-team/keras/blob/07e13740fd181fc3ddec7d9a594d8a08666645f6/keras/utils/metrics_utils.py#L844
y_pred = tf.convert_to_tensor(y_pred)
threshold = tf.cast(threshold, y_pred.dtype)
y_pred = tf.cast(y_pred > threshold, y_pred.dtype)
binary_matches = tf.cast(tf.equal(y_true, y_pred), K.floatx())
tf.reduce_mean(binary_matches, axis=-1)

In [112]:
m = tf.keras.metrics.BinaryAccuracy()
r = Reduce(
    MeanMetricWrapper(m._fn).reduction,
    MeanMetricWrapper(m._fn).name
)
total = r.total
count = r.count

# https://github.com/keras-team/keras/blob/07e13740fd181fc3ddec7d9a594d8a08666645f6/keras/metrics/base_metric.py#L401
# update_state cont.
[values], sample_weight = \
    metrics_utils.ragged_assert_compatible_and_get_flat_values(
        [values], sample_weight)
values = tf.cast(values, 'float32')

# Incorporate weight
if sample_weight is not None:
    sample_weight = tf.cast(sample_weight, 'float32')
    
    values, _, sample_weight = losses_utils.squeeze_or_expand_dimensions(
            values, sample_weight=sample_weight)
    
    sample_weight = tf.__internal__.ops.broadcast_weights(
                sample_weight, values)
    
    values = tf.multiply(values, sample_weight)

value_sum = tf.reduce_sum(values)

with tf.control_dependencies([value_sum]):
    update_total_op = total.assign_add(value_sum)
    
if sample_weight is None:
    num_values = tf.cast(tf.size(values), 'float32')
else:
    num_values = tf.reduce_sum(sample_weight)
    
with tf.control_dependencies([update_total_op]):
    count.assign_add(num_values)

# result
tf.math.divide_no_nan(total, count)

<tf.Tensor: shape=(), dtype=float32, numpy=0.75>

In [ ]:
total

In [ ]:
count

In [ ]:
def compute_weight(y_true, y_pred):
    y_true = tf.convert_to_tensor(y_true)
    y_pred = tf.convert_to_tensor(y_pred)

    y_flat = y_true
    if y_true.shape.ndims == y_pred.shape.ndims:
        y_flat = tf.squeeze(y_flat, axis=[-1])
    y_true_int = tf.cast(y_flat, tf.int32)
    # print(y_true_int)

    cls_counts = tf.math.bincount(y_true_int)
    # print(cls_counts)
    cls_counts = tf.math.reciprocal_no_nan(tf.cast(cls_counts, 'float32'))
    # print(cls_counts)
    weight = tf.gather(cls_counts, y_true_int)
    # print(f'weight: {weight}')
    ratio = cls_counts
    return weight, ratio

In [ ]:
# m = BalancedSparseCategoricalAccuracy()
m = tf.keras.metrics.BinaryAccuracy()
y_trues = []
y_preds = []
weights = []
for input, label in single_step_window.test(REMOVE_WEARING_OFF_IN_PREVIOUS_STEP):
    y_true = label
    y_trues.append(y_true.numpy())
    print(f'Label: { [y_t for y_t in y_true.numpy().reshape(y_true.numpy().shape[0]) ]}')
    
    y_pred = linear(input)
    y_preds.append(y_pred.numpy())
    print(f'Predictions: {  [y_p for y_p in y_pred.numpy().reshape(y_pred.numpy().shape[0]) ]}')
    
    THRESHOLD = 0.5
    cm = confusion_matrix(y_true.numpy().reshape(y_true.shape[0]),
                          y_pred.numpy().reshape(y_pred.shape[0]) > THRESHOLD)
    print(f'CM: {cm}')
    
    weight, ratio = compute_weight(y_true, y_pred)
    weights.append(weight)
    print(ratio)
    
    m.update_state(y_true, y_pred, sample_weight=weight)
    print(f'result: {m.result().numpy()}')
    print()
    
# y_trues = np.concatenate(y_trues)
# y_preds = np.concatenate(y_preds)